In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import squarify
import seaborn as sns
from surprise import accuracy
from surprise.model_selection.validation import cross_validate
from surprise.dataset import Dataset
from surprise.reader import Reader
from surprise import SVD, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split
import random

## Calling the Saved Dataset

In [2]:
course_dataset = pd.read_csv("data/courseData.csv", encoding= 'unicode_escape')
course_dataset.head()

,Unnamed: 0,sr_,degree_1,degree_1_specializations,campus,key_skills,key_skills_str
0,0,1001,B.E.,Mechanical,MITCOE,['CATIA'],CATIA
1,1,1002,B.E.,Mechanical,MITCOE,['CATIA'],CATIA
2,2,1003,B.E.,Mechanical,MITAOE,['CATIA'],CATIA
3,3,1004,B.E.,Mechanical,MITCOE,['CATIA'],CATIA
4,4,1005,B.E.,Mechanical,MITCOE,['CATIA'],CATIA


In [3]:
user_dataset = pd.read_csv("data/userData.csv", encoding= 'unicode_escape')
user_dataset.head()

,Unnamed: 0,userid,degree_1,degree_1_specializations,known_languages,key_skills,career_objective,key_skills_str
0,0,1001,B.E.,Computer Science & Engineering,"['English', ' Marathi', ' Hindi']","['C', ' Java', ' Keras', ' Flask', ' Deep Lear...",Computer Engineering student with good technic...,"C, Java, Keras, Flask, Deep Learning, Sel..."
1,1,1002,B.E.,Computer Science & Engineering,['Hindi English'],"['Java', ' Neural Networks', ' AI', ' Python',...",Interested in working under company offering A...,"Java, Neural Networks, AI, Python, Html5, ..."
2,2,1003,B.E.,Computer Science & Engineering,['Missing'],['Missing'],Missing,Missing
3,3,1004,B.E.,Computer Science & Engineering,"['English', ' Hindi', ' Marathi', ' Marwari']","['XML', ' C', ' Java', ' Data Structures', ' P...",Currently a final year student of Computer Eng...,"XML, C, Java, Data Structures, Python, Mo..."
4,4,1005,B.E.,Computer Science & Engineering,"['English', 'Hindi ', 'Kashmiri ', ' Urdu']","['XML', ' Word', ' Data Structures', ' Communi...",To have a growth oriented and challenging care...,"XML, Word, Data Structures, Communication, ..."


In [4]:
ratings_df = pd.read_csv("data/ratingData.csv", encoding= 'unicode_escape')
ratings_df.head()

,course_id,user_id,rating
0,2001,1001,5
1,2001,1002,3
2,2001,1003,1
3,2001,1004,0
4,2001,1005,2


## Content Based Filtering

In [5]:
#https://github.com/ry05/couReco/blob/master/recommender.py
#https://github.com/jalajthanaki/Movie_recommendation_engine/blob/master/Movie_recommendation_engine.ipynb

### Based on Description and Taglines
### https://github.com/jalajthanaki/Movie_recommendation_engine/blob/master/Movie_recommendation_engine.ipynb

In [6]:
course_dataset.head(2)

,Unnamed: 0,sr_,degree_1,degree_1_specializations,campus,key_skills,key_skills_str
0,0,1001,B.E.,Mechanical,MITCOE,['CATIA'],CATIA
1,1,1002,B.E.,Mechanical,MITCOE,['CATIA'],CATIA


In [7]:
user_dataset.head(2)

,Unnamed: 0,userid,degree_1,degree_1_specializations,known_languages,key_skills,career_objective,key_skills_str
0,0,1001,B.E.,Computer Science & Engineering,"['English', ' Marathi', ' Hindi']","['C', ' Java', ' Keras', ' Flask', ' Deep Lear...",Computer Engineering student with good technic...,"C, Java, Keras, Flask, Deep Learning, Sel..."
1,1,1002,B.E.,Computer Science & Engineering,['Hindi English'],"['Java', ' Neural Networks', ' AI', ' Python',...",Interested in working under company offering A...,"Java, Neural Networks, AI, Python, Html5, ..."


### User Dataset - Key Skills 

In [8]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(user_dataset['key_skills_str'].values.astype('U'))
tfidf_matrix.shape


(1097, 2065)

In [9]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.02408797, 0.        , ..., 0.06121443, 0.01629229,
       0.007909  ])

In [10]:
user_dataset = user_dataset.reset_index()
titles = user_dataset['userid']
indices = pd.Series(user_dataset.index, index=user_dataset['userid'])
indices.head(2)

userid
1001    0
1002    1
dtype: int64

In [11]:
def get_recommendations(title): #this is getting similar users based on their career objectives???
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

get_recommendations("1001").head(10)

894     1847
996     1946
981     1931
256     1231
868     1821
970     1920
1089    2039
714     1667
417     1375
493     1444
Name: userid, dtype: object

In [12]:
get_recommendations("1847").head(10)

996    1946
981    1931
0      1001
417    1375
256    1231
322    1292
714    1667
112    1104
201    1182
521    1473
Name: userid, dtype: object

In [13]:
get_recommendations("1946").head(10)

996    1946
981    1931
0      1001
417    1375
256    1231
322    1292
714    1667
112    1104
201    1182
521    1473
Name: userid, dtype: object

###  User Dataset - Career Objective

In [14]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(user_dataset['career_objective'].values.astype('U'))
tfidf_matrix.shape

(1097, 4673)

In [15]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]


array([1.        , 0.        , 0.        , ..., 0.00592012, 0.        ,
       0.0596521 ])

In [16]:
user_dataset = user_dataset.reset_index()
titles = user_dataset['userid']
indices = pd.Series(user_dataset.index, index=user_dataset['userid'])
indices.head(2)

userid
1001    0
1002    1
dtype: int64

In [17]:
def get_recommendations(title): #this is getting similar users based on their career objectives???
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

get_recommendations("1001").head(10)

894     1847
996     1946
93      1087
110     1102
180     1161
20      1021
914     1867
602     1555
801     1754
1022    1972
Name: userid, dtype: object

In [18]:
user_dataset["career_objective"].loc[user_dataset['userid'] == '1001']

0    Computer Engineering student with good technic...
Name: career_objective, dtype: object

In [19]:
user_dataset.loc[user_dataset['userid'] == '1847']

,level_0,index,Unnamed: 0,userid,degree_1,degree_1_specializations,known_languages,key_skills,career_objective,key_skills_str
894,894,894,842,1847,B.E.,Computer Science & Engineering,"['English', ' Marathi', ' Hindi']","['C', ' Java', ' Keras', ' Flask', ' Deep Lear...",Computer Engineering student with good technic...,"C, Java, Keras, Flask, Deep Learning, Sel..."


In [20]:
user_dataset.loc[user_dataset['userid'] == '1087']

,level_0,index,Unnamed: 0,userid,degree_1,degree_1_specializations,known_languages,key_skills,career_objective,key_skills_str
93,93,93,86,1087,B.E.,Computer Science & Engineering,"['English', ' Marathi ', ' Hindi']","['Java', ' Python', ' Machine Learning', ' CPP...",Dedicated and passionate computer engineering ...,"Java, Python, Machine Learning, CPP, Andro..."


### User Dataset - career objective and key skills


In [30]:
#creating the dataset

cf_dataset = pd.DataFrame()
cf_dataset['userid'] = user_dataset['userid']
cf_dataset['career_objective'] = user_dataset['career_objective']
cf_dataset['key_skills_str'] = user_dataset['key_skills_str']
cf_dataset.head(5)

,userid,career_objective,key_skills_str
0,1001,Computer Engineering student with good technic...,"C, Java, Keras, Flask, Deep Learning, Sel..."
1,1002,Interested in working under company offering A...,"Java, Neural Networks, AI, Python, Html5, ..."
2,1003,Missing,Missing
3,1004,Currently a final year student of Computer Eng...,"XML, C, Java, Data Structures, Python, Mo..."
4,1005,To have a growth oriented and challenging care...,"XML, Word, Data Structures, Communication, ..."


In [29]:
s = cf_dataset.apply(lambda x: pd.Series(x['career_objective']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'career_objective'
s = s.value_counts()
s[:5]

Missing                                                                                                                                                                                                                                      570
To secure a position where I can efficiently contribute my skills and abilities to the growth of the organization and build my professional career.                                                                                            7
To pursue a challenging career and be a part of progressive organization that gives a scope to enhance my knowledge and utilizing my skills towards the growth of the organization.                                                            5
Looking for a challenging role in a reputable organization to utilize my technical, database, and management skills for the growth of the organization as well as to enhance my knowledge about new and emerging trends in the IT sector.      4
Computer Engineering student with go

In [34]:
cf_dataset['all'] = cf_dataset['career_objective'] + cf_dataset['key_skills_str']
cf_dataset

,userid,career_objective,key_skills_str,all
0,1001,Computer Engineering student with good technic...,"C, Java, Keras, Flask, Deep Learning, Sel...",Computer Engineering student with good technic...
1,1002,Interested in working under company offering A...,"Java, Neural Networks, AI, Python, Html5, ...",Interested in working under company offering A...
2,1003,Missing,Missing,MissingMissing
3,1004,Currently a final year student of Computer Eng...,"XML, C, Java, Data Structures, Python, Mo...",Currently a final year student of Computer Eng...
4,1005,To have a growth oriented and challenging care...,"XML, Word, Data Structures, Communication, ...",To have a growth oriented and challenging care...
...,...,...,...,...
1092,2042,i have to be carrer in programming and after t...,"java, database, html, OOPs, Core Java, MySQL",i have to be carrer in programming and after t...
1093,2043,To work in an Industry with a professional wor...,"Ability-to-cope-up-with-different-situation., ...",To work in an Industry with a professional wor...
1094,2044,To pursue a highly challenging and creative ca...,"C, Java, cpp, HTML, Basic-Python, MySQL",To pursue a highly challenging and creative ca...
1095,2045,To prove myself dedicated worthful and energet...,"C, Drupal-(CMS), Bootstrap, Wordpress-(CMS)...",To prove myself dedicated worthful and energet...


In [45]:
#vectorize 
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(cf_dataset['all'].values.astype('U'))

#cosine similarity using linear kernel
cosine_sim = cosine_similarity(count_matrix, count_matrix)

#
cf_dataset = cf_dataset.reset_index()
users = cf_dataset['userid']
indices = pd.Series(cf_dataset.index, index=cf_dataset['userid'])


#function to get recommendations
def get_recommendations(user): #this is getting similar users based on their career objectives???
    idx = indices[user]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    user_indices = [i[0] for i in sim_scores]
    return users.iloc[user_indices]

In [46]:
get_recommendations("1001").head(10)

894    1847
996    1946
93     1087
256    1231
201    1182
180    1161
91     1085
464    1416
981    1931
165    1146
Name: userid, dtype: object

In [49]:
get_recommendations("1847").head(10)

996    1946
0      1001
93     1087
981    1931
110    1102
256    1231
180    1161
91     1085
165    1146
201    1182
Name: userid, dtype: object

In [50]:
get_recommendations("1946").head(10)

996    1946
0      1001
93     1087
981    1931
110    1102
256    1231
180    1161
91     1085
165    1146
201    1182
Name: userid, dtype: object

In [52]:
cf_dataset["key_skills_str"].loc[cf_dataset['userid'] == '1001']

0    C,  Java,  Keras,  Flask,  Deep Learning,  Sel...
Name: key_skills_str, dtype: object

In [57]:
cf_dataset["all"].loc[cf_dataset['userid'] == '1847']

894    Computer Engineering student with good technic...
Name: all, dtype: object

In [58]:
cf_dataset["all"].loc[cf_dataset['userid'] == '1946']

996    Computer Engineering student with good technic...
Name: all, dtype: object

In [62]:
cf_dataset["all"].loc[cf_dataset['userid'] == '1231']

256    To enhance my expertise in the field of softwa...
Name: all, dtype: object

Upon checking the above, the filter is checking all the users' career objectives and key skill. And returns similar users. 

User 1001 has career objective a and key skills list b. 

User 1847 (the first recommendation for user 1001) has career objective a and key skills list b.

User 1847 (the second recommendation for user 1001) has career objective a and key skills list b.

User 1231 (the fourth recommendation for user 1001) has career objective NEW and key skills list NEW with some matches like Keras.

### Course Dataset -  degree 1, degree 1 specialization and key skills

In [64]:
#creating the dataset

cf_dataset2 = pd.DataFrame()
cf_dataset2['sr_'] = course_dataset['sr_']
cf_dataset2['degree_1'] = course_dataset['degree_1']
cf_dataset2['degree_1_specializations'] = course_dataset['degree_1_specializations']
cf_dataset2['key_skills_str'] = course_dataset['key_skills_str']
cf_dataset2.head(5)

,sr_,degree_1,degree_1_specializations,key_skills_str
0,1001,B.E.,Mechanical,CATIA
1,1002,B.E.,Mechanical,CATIA
2,1003,B.E.,Mechanical,CATIA
3,1004,B.E.,Mechanical,CATIA
4,1005,B.E.,Mechanical,CATIA


In [65]:
cf_dataset2['all'] = cf_dataset2['degree_1'] + cf_dataset2['degree_1_specializations'] + cf_dataset2['key_skills_str']
cf_dataset2

,sr_,degree_1,degree_1_specializations,key_skills_str,all
0,1001,B.E.,Mechanical,CATIA,B.E.MechanicalCATIA
1,1002,B.E.,Mechanical,CATIA,B.E.MechanicalCATIA
2,1003,B.E.,Mechanical,CATIA,B.E.MechanicalCATIA
3,1004,B.E.,Mechanical,CATIA,B.E.MechanicalCATIA
4,1005,B.E.,Mechanical,CATIA,B.E.MechanicalCATIA
...,...,...,...,...,...
9995,10996,B.E.,Electronics Telecommunication Engineering,"EmbeddedC, MATLAB, Cprogramming, Keil",B.E.Electronics Telecommunication Engineering...
9996,10997,B.E.,Electronics Telecommunication Engineering,"EmbeddedC, MATLAB, Cprogramming, Keil",B.E.Electronics Telecommunication Engineering...
9997,10998,M TeCh,Electronics Telecommunication Engineering,"EmbeddedC, MATLAB, Cprogramming, Keil",M TeCh Electronics Telecommunication Engineer...
9998,10999,B.E.,Electronics Telecommunication Engineering,"AmazonWebServiCes, C CPP, Arduino, MongoDB, Li...",B.E.Electronics Telecommunication Engineering...


In [66]:
#vectorize 
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(cf_dataset2['all'].values.astype('U'))

#cosine similarity using linear kernel
cosine_sim = cosine_similarity(count_matrix, count_matrix)

#
cf_dataset2 = cf_dataset2.reset_index()
courses = cf_dataset2['sr_']
indices = pd.Series(cf_dataset2.index, index=cf_dataset2['sr_'])


#function to get recommendations
def get_recommendations(course): 
    idx = indices[course]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    course_indices = [i[0] for i in sim_scores]
    return courses.iloc[course_indices]

In [68]:
get_recommendations(1001).head(10)

1     1002
2     1003
3     1004
4     1005
5     1006
6     1007
7     1008
8     1009
9     1010
10    1011
Name: sr_, dtype: int64

In [77]:
get_recommendations(1003).head(10)

1     1002
2     1003
3     1004
4     1005
5     1006
6     1007
7     1008
8     1009
9     1010
10    1011
Name: sr_, dtype: int64

In [72]:
print(cf_dataset2["all"].loc[cf_dataset2['sr_'] == 1001])  #key_skills_str

0    B.E.MechanicalCATIA
Name: all, dtype: object
0    CATIA
Name: key_skills_str, dtype: object


In [75]:
print(cf_dataset2["all"].loc[cf_dataset2['sr_'] == 1002])
print(cf_dataset2["all"].loc[cf_dataset2['sr_'] == 1003])

1    B.E.MechanicalCATIA
Name: all, dtype: object
2    B.E.MechanicalCATIA
Name: all, dtype: object


In [78]:
#trying a different course
get_recommendations(10996).head(10)

172    1173
209    1210
212    1213
213    1214
214    1215
225    1226
226    1227
230    1231
231    1232
435    1436
Name: sr_, dtype: int64

In [82]:
print(cf_dataset2["all"].loc[cf_dataset2['sr_'] == 10996])
print(cf_dataset2["key_skills_str"].loc[cf_dataset2['sr_'] == 1173])
print(cf_dataset2["key_skills_str"].loc[cf_dataset2['sr_'] == 1210])

9995    B.E.Electronics  Telecommunication Engineering...
Name: all, dtype: object
172    EmbeddedC, MATLAB, Cprogramming, Keil
Name: key_skills_str, dtype: object
209    EmbeddedC, MATLAB, Cprogramming, Keil
Name: key_skills_str, dtype: object


They are also accurate in the course dataset. Only the degree, degree specializations and key skils are basically the same.

Try to implement the removal of the course we are looking for from the recommendations. As seen above in the 1003 course search, the thing we search for is sometimes given back

### Course Dataset - degree 1, degree 1 specialization and key skills

In [21]:
course_dataset.head(2)

,Unnamed: 0,sr_,degree_1,degree_1_specializations,campus,key_skills,key_skills_str
0,0,1001,B.E.,Mechanical,MITCOE,['CATIA'],CATIA
1,1,1002,B.E.,Mechanical,MITCOE,['CATIA'],CATIA


In [22]:
df = pd.DataFrame()
df["details"] = course_dataset["degree_1"] + course_dataset["degree_1_specializations"] + course_dataset["key_skills_str"]

In [23]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['details'].values.astype('U'))

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

df = df.reset_index()
titles = course_dataset['sr_']
indices = pd.Series(course_dataset.index, index=course_dataset['sr_'])
indices.head(2)

sr_
1001    0
1002    1
dtype: int64

In [24]:
def get_recommendations(title): #this is getting similar users based on their career objectives???
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

get_recommendations("1002").head(10)

KeyError: '1002'

## Another Methodology

In [25]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVR

In [ ]:
# Splitting the ratings dataset into the feature set (X) and target labels (y)
X = ratings_df.drop(columns='Rangering')
y = ratings_df["Rangering"].values  # The movie ratings are the target variables we want to predict

# Preparing train, validation and test datasets.
# I have chosen a split ratio of 70%, 15%, 15%, because I want a somewhat large training set at the cost of a
# smaller validation and test set. I do not think that a smaller validation (or test) dataset will negatively
# impact the generalization ability of the chosen models, because I am only using rather simple ML models
# with few hyperparamaters.
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=101)

# Creating a complete training dataset with X_train and y_train
train_df = X_train.copy()
train_df["Rangering"] = y_train

train_df

In [26]:
# ----------- CREATING THE DIFFERENT CONTENT-BASED FILTERING MODELS ----------- #
# Assigning the different machine learning algorithms to be implemented in the models (incl. hyperparameters) to a dictionary
ml_algorithms = {"Linear regression": LinearRegression(), 
                 "Lasso": Lasso(alpha=1.0, max_iter=10000), 
                 "KNN_7": KNeighborsRegressor(n_neighbors=7),
                 "RFR": RandomForestRegressor(n_estimators=1000, n_jobs=3, max_features="auto", random_state=0),
                 "SVR": SVR(C=1.0)}

# Saving lists that I later use to construct a dataframe containing the performances of the models
CBF_models_listed = []
RMSE_CBF_listed = []

# For every machine learning algorithm in the dictionary:
for name, ml_alg in ml_algorithms.items():
    # Create an empty list for predictions
    CBF_predictions = []

    # For each user in the training dataset:
    for i, x in enumerate(x_train_listed):
        # Fit a machine learning model
        ml_alg.fit(x_train_listed[i], y_train_listed[i])
        # Predict all the ratings for this user for all movies
        prediction = ml_alg.predict(all_movies)
        prediction = np.clip(prediction, 1, 5)  # Predictions must be minimum 1, maximum 5
        # Append all the predictions to the predictions list
        CBF_predictions.append(prediction)

    # Create a dataframe with the predictions
    df_predict = pd.DataFrame(CBF_predictions, index=user_ids, columns=movie_ids)

    # Create a dataframe with only the predictions for the movies-user combinations that appear in the validation set
    num_actual = validation_matrix.to_numpy().flatten()[validation_matrix.notna().to_numpy().flatten()]
    num_predict = df_predict.to_numpy().flatten()[validation_matrix.notna().to_numpy().flatten()]

    # Calculate the RMSE for the content-based filtering model and add the result to the lists
    RMSE_CBF_listed.append(sqrt(mean_squared_error(num_predict, num_actual)))
    CBF_models_listed.append(name)


# Printing the results
RMSE_CBF_df = pd.DataFrame({"Model": CBF_models_listed, "RMSE": RMSE_CBF_listed})
print("RMSE of different content-based filtering models without the year of release feature:")
RMSE_CBF_df

NameError: name 'x_train_listed' is not defined